In [8]:
import pandas as pd 
from matplotlib import pyplot as plt
import numpy as np
import math

def Rad(d):
    return d * math.pi / 180

def Geodist(point1,point2):
    radLat1 = Rad(point1[1])
    radLat2 = Rad(point2[1])
    delta_lon = Rad(point1[0] - point2[0])
    top_1 = math.cos(radLat2) * math.sin(delta_lon)
    top_2 = math.cos(radLat1) * math.sin(radLat2) - math.sin(radLat1) * math.cos(radLat2) * math.cos(delta_lon)
    top = math.sqrt(top_1 * top_1 + top_2 * top_2)
    bottom = math.sin(radLat1) * math.sin(radLat2) + math.cos(radLat1) * math.cos(radLat2) * math.cos(delta_lon)
    delta_sigma = math.atan2(top, bottom)
    distance = delta_sigma * 6378137.0

    return round(distance,3)


#2.点弦距离
def get_vertical_dist(pointA,pointB,pointX):
    a=math.fabs(Geodist(pointA,pointB))

    #当弦两端重合时,点到弦的距离变为点间距离
    if a==0:
        return math.fabs(Geodist(pointA,pointX))

    b=math.fabs(Geodist(pointA,pointX))
    c=math.fabs(Geodist(pointB,pointX))
    p=(a+b+c)/2
    S=math.sqrt(math.fabs(p*(p-a)*(p-b)*(p-c)))

    vertical_dist=S*2/a

    return vertical_dist

def get_MeanErr(point_list,output_point_list):
    Err=0

    start_index=0
    end_index=len(output_point_list)-1

    while(start_index<end_index):        #遍历所有关键点
        #选取两相邻关键点
        pointA_id=int(output_point_list[start_index][2])
        pointB_id=int(output_point_list[start_index+1][2])

        id=pointA_id+1        #工作指针,用于遍历非关键点
        while(id<pointB_id):        #遍历两关键点之间的非关键点
            Err+=get_vertical_dist(output_point_list[start_index],output_point_list[start_index+1],point_list[id])
            id+=1

        start_index+=1

    return Err/len(point_list)

# def presift(point_list):
#     lim=10e-7
#     flag=0 #记录前面速度，0表示前面速度为0
#     for item in point_list:
#         if item[3]<lim and flag==0:
#             #此时速度为0，并且前面的速度为0，删去此点
#             #保留
#             point_list.remove(item)
#         # 速度不为0，不删去
#         #速度为0，前面速度不为0，不删去
#     return


In [9]:
#point_list格式为lon，lat，id
def slidingwindows(point_list,output_point_list,Dmax):
    s_index=0 #记录窗口起点
    e_index=2 #记录窗口终点
    end_index=len(point_list)-1 #这个数组的最后一个索引
    output_point_list.append(point_list[s_index])
    output_point_list.append(point_list[end_index])
    while e_index<end_index:
        pl=point_list[e_index] #窗口中最后一个点
        pf=point_list[s_index] # 窗口中第一个点
        pi=point_list[e_index-1] #窗口中倒数第二个点
        if (pl[0]-pi[0])*(pi[0]-pf[0])>=0 and (pl[1]-pi[1])*(pi[1]-pf[1])>=0:
            e_index+=1
        else:
            index=s_index+1        #
            max_vertical_dist=0        #窗口中最大距离

            while(index<e_index):
                cur_vertical_dist=get_vertical_dist(point_list[s_index],point_list[e_index],point_list[index])
                if cur_vertical_dist>max_vertical_dist:
                    max_vertical_dist=cur_vertical_dist
                index+=1

            if max_vertical_dist>Dmax:
                output_point_list.append(point_list[e_index-1])
                s_index=e_index
                e_index=s_index+2
            else: e_index+=1
    return 

In [13]:
path="D:\\数模选拔\\data\\"
p=path+"20161018 (35).txt"
point_list=[]
output_point_list=[]
data=pd.read_csv(p,sep='\\s+',header=None)
data.drop(data.columns[9],axis=1)
    #读取数据
date=data.loc[:,0]
time=data.loc[:,1]
id=data.loc[:,2]
lon_now=data.loc[:,3]
lat_now=data.loc[:,4]
lon_pre=data.loc[:,5]
lat_pre=data.loc[:,6]
v=data.loc[:,7]
angle=data.loc[:,8]

for i in range(len(data)):
    longitude=lon_now[i]
    latitude=lat_now[i]
    #ve=v[i]
    point_list.append((longitude,latitude,i))

slidingwindows(point_list,output_point_list,Dmax=0.01)

output_point_list=list(set(output_point_list))        #去除递归引入的冗余数据
output_point_list=sorted(output_point_list,key=lambda x:x[2])        #按照id排序

    #将压缩数据写入输出文件
fd=open(r".\output.txt",'w')
for point in output_point_list:
    fd.write("{},{},{}\n".format(point[2],point[0],point[1]))
fd.close()
print("compression rate={}/{}={}".format(len(output_point_list),len(point_list),len(output_point_list)/len(point_list)))
print("mean error:{}".format(get_MeanErr(point_list,output_point_list)))

uncompressed=[[],[]]
for point in point_list[:]:#point_list
    uncompressed[0].append(point[0])
    uncompressed[1].append(point[1])
plt.subplot(1,2,1)
plt.scatter(uncompressed[0],uncompressed[1],s=1,color="green")
plt.xlabel("longitude")
plt.ylabel("latitude")
plt.title("uncompressed")
compressed=[[],[]]
for point in output_point_list:#output_point_list
    compressed[0].append(point[0])
    compressed[1].append(point[1])
plt.subplot(1,2,2)
plt.plot(compressed[0],compressed[1],markersize=1,color='r')
plt.xlabel("longitude")
plt.ylabel("latitude")
plt.title("compressed")
plt.show()

compression rate=73/492=0.1483739837398374
mean error:331.3494245231648
